<a href="https://colab.research.google.com/github/mmedhat2/Projects/blob/branch1/mysongs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd


# Set up client credentials
client_id = '74b2f8f7c60145bfb55ab8e18a1e5b6f'
client_secret = '1b5171a928934677bcd442c98dcb0eec'

# Authenticate with Spotify Web API
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)




In [ ]:
def call_playlist(creator, playlist_id,fields=None, limit=1000, offset=0, market=None):
    
    results = sp.user_playlist_tracks(creator,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks    

    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    playlist_tracks_first_artists = []
    playlist_tracks_first_release_date = []

    for i in range(len(results)):
        if i == 0:
            playlist_tracks_id = results[i]['track']['id']
            playlist_tracks_titles = results[i]['track']['name']
            playlist_tracks_first_release_date = results[i]['track']['album']['release_date']

            artist_list = []
            for artist in results[i]['track']['artists']:
                artist_list= artist['name']
            playlist_tracks_artists = artist_list

            features = sp.audio_features(playlist_tracks_id)
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['title'] = playlist_tracks_titles
            features_df['artist'] = playlist_tracks_artists
            features_df = features_df[['id', 'title', 'artist']]
            continue
        else:
            try:
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list= artist['name']
                playlist_tracks_artists = artist_list
                features = sp.audio_features(playlist_tracks_id)
                new_row = {'id':[playlist_tracks_id],
               'title':[playlist_tracks_titles],
               'all_artists':[playlist_tracks_artists],
               
               }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index = True)
            except:
                continue
                
    return features_df



In [ ]:
spotify = call_playlist('mohamed_medhat55','1gTHjxjQBzUvdnhnCHy2HY')

In [ ]:
spotify['Artist'] = spotify[['artist', 'all_artists']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
spotify = spotify.drop(columns=['id','all_artists','artist'],axis=1)

In [ ]:
spotify

,title,Artist
0,Limbo,Freddie Dredd
1,GTG,Freddie Dredd
2,All Alone,Freddie Dredd
3,Even If It's a Lie - Demo,Matt Maltese
4,Shoe,Matt Maltese
...,...,...
594,Rockstar Made,Playboi Carti
595,On That Time,Playboi Carti
596,Stop Breathing,Playboi Carti
597,veil,須田景凪


In [ ]:
anghami = pd.read_csv('anghami.csv')
sp

In [ ]:
spotify[~spotify['title'].isin(anghami['Song Name'])]

,title,Artist
3,Even If It's a Lie - Demo,Matt Maltese
11,On Time (with John Legend),John Legend
12,I Can't Save You (Interlude) [with Future & fe...,Don Toliver
13,Feel The Fiyaaaah (with A$AP Rocky & feat. Tak...,Takeoff
14,Creepin' (with The Weeknd & 21 Savage),21 Savage
...,...,...
586,Yesterday - Remastered 2009,The Beatles
587,It Won't Be Long - Remastered 2009,The Beatles
588,خصمي شفرتو,مودي امين
589,مسا مني ليكوا,Essam Sasa
